# Perform mps+dmft calculation on Anderson impurity model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import linalg 

In [ ]:
# plot setting 
import matplotlib as mpl
from matplotlib import rcParams
from matplotlib import rc

plt.style.use('default')

rcParams[ 'axes.titlesize'] = 25
rcParams[ 'axes.labelsize'] = 25
rcParams[ 'lines.markersize'] = 5
rcParams[ 'xtick.labelsize'] = 22
rcParams[ 'ytick.labelsize'] = 22
rcParams[ 'legend.fontsize'] = 25
rcParams[ 'legend.frameon'] = True

# mpl.rc('font', family='sans-serif')
mpl.rc('text', usetex=True)

mpl.rcParams['text.latex.preamble'] = [
       r'\usepackage{amsmath}',
       r'\usepackage{helvet}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
    
]  


In [ ]:
# import sys
# sys.path.insert(0,"../input/mps-imp-wf")

In [ ]:
from imp_chain_ham import *
from mps import *
from TEBD import TEBD_class # main solver
from parameter import parameter # parameter

from bath_generator import spectral

# Test one-chain free fermion

In [ ]:

# --------------------------------------------------------
#                    parameter
# --------------------------------------------------------
# Set interaction four orbital
Dmax = 250 # max bound dimension

num_of_imp = int(4) # four imp
U_hubbard = 0.0 # density interaction
JH         = 0.0 # Hund's

U ={ '12':U_hubbard, '13':U_hubbard - 0.5 * JH, '14':U_hubbard, '23':U_hubbard, '24':U_hubbard - 0.5 * JH, '34':U_hubbard }
J = -0.5 * JH
ed = np.ones( num_of_imp ) * ( -1.5 * U_hubbard ) #fix half filling
E0 = 0.0
para = parameter(Dmax, ed, U, J,E0) # import para
para.tau = 5.0

# get bath
const_spec = spectral()
const_spec.constant_dos(600)
[vl, el, de] = const_spec.get_bath_coeff(200)
[w, rc] = const_spec.get_dos_from_para(vl, el, de, 1.0)
# vl = np.zeros(el.shape)
# el = np.zeros(vl.shape)


In [ ]:
# --------------------------------------------------------
#                    initialization
# --------------------------------------------------------
# definte Ham
chain_ham = []
for i in range(0,4):
    ham = single_chain_ham(el, vl, 0.0)
    chain_ham.append(ham)

# chain mps
L_bath = len(el)
chain_mpo = []
for i in range(0,4):
    if( i% 2 == 0):
        if_bath_sign = True 
    else:
        if_bath_sign = False
    mps_ = mps(para, L_bath+1, if_bath_sign)
    chain_mpo.append(mps_)
# TEBD solver
tebd_ = TEBD_class(para)
int_gate = int_ham(para,para.tau/2.0)
tebd_.init(para)
tebd_.import_chain_ham(chain_ham)
tebd_.import_int_gate(int_gate, para.tau/2.0)

In [ ]:
print(tebd_.chain_mpo[0].D)

In [ ]:

# --------------------------------------------------------
#                    one chain fermion
#                     real time evolution
# --------------------------------------------------------
N_tau = 500
para.tau = 2.0
En = 9999
int_gate.update_dt(para.tau/2.0)
for i in range(0, N_tau):
    if( i%10 == 0 and i!=0 and para.tau > 0.02):
        para.tau = para.tau/2.0
    
    if( i%2 ==0):
        En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = True)
        print("gnd energy = ",En_new)
        diff = np.abs( En - En_new )
        En = En_new
        if( diff < 1e-8 and i > N_tau/2):
            if( para.tau < 0.02 ):
                print("Solved")
                break 
            else:
                para.tau = para.tau/2.0       
    else:
         En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = False)

In [ ]:
tebd_.save_gnd()

In [ ]:
tebd_.load_gnd()
imp_ind = 2

Nt = 10
t = []
gt_num = []
para.t = 0.02



tebd_.act_d_dag(imp_ind)


for i in range(0,Nt):
    t.append(i * para.t)
        
        

    gt = tebd_.trace_with_d(imp_ind)
    print("------------")
#     gt = chain_mpo[0].trace_()
    print("i, gt",i, gt)
    gt_num.append(gt)
    tebd_.time_evolution(1j * para.t, if_calculate_gnd_en = False)
#     tebd_.one_step_chain_projection(tebd_.chain_mpo[0], tebd_.chain_ham[0], 1j*para.t )


# tebd_.load_gnd()
# imp_ind = 0

# Nt = 10
# t = []
# gt_num = []
# para.t = 0.02





# for i in range(0,Nt):
#     t.append(i * para.t)
        
        

#     gt = tebd_.trace()
#     print("------------")
# #     gt = chain_mpo[0].trace_()
#     print("i, gt",i, gt)
#     print(np.exp( -1j * ( t[i] *(-5.952135))) )
#     gt_num.append(gt)
#     tebd_.time_evolution(1j * para.t, if_calculate_gnd_en = False)
# #     tebd_.one_step_chain_projection(tebd_.chain_mpo[0], tebd_.chain_ham[0], 1j*para.t )





In [ ]:
def get_eig( vl, el):
    # get eigen value of the ham 
    nc = len(vl)
    ham = np.zeros( (nc+1,nc+1), dtype = np.float32)
    np.fill_diagonal( ham[1:,1:], el)
    ham[0,1:] = vl 
    ham[1:,0] = vl 
    ham[0,0]  = 0.00
    eg,eg_v = linalg.eig(ham)
   
    return [eg,eg_v] 

def cdag_c(t, eg,eg_v):
    # <psi | d e^{-iHt} d^dag |psi>
    vv = eg_v * np.conj(eg_v)     
    vv = vv[0,:]*(np.exp(-1j * eg * t)) 
    vv = vv*(eg > 0.0)
    gt = np.sum(vv)
    en = np.sum(eg * ( eg < 0.0) )
    en = np.exp( -1j * t * en )
    gt = gt * en 
#     gt = gt *  np.exp(-1j*t * np.sum( eg * (eg<0.0) ) ) 
#     print(gt)
    return gt

eg,eg_v = get_eig(vl,el)
print("Gnd energy = ",4.0 * np.sum(eg * (eg < 0.0 )))
eg_each_chain = np.sum(eg * (eg < 0.0 )) 

In [ ]:
gt_ana = []
for t_it in t:
    gt =  cdag_c(t_it, eg, eg_v) 
    gt = gt * np.exp(- 1j * t_it * eg_each_chain * 3.0)
#     gt =  np.exp(-4j * t_it * eg_each_chain)
    gt_ana.append( gt )

In [ ]:
t = np.asarray(t)
gt_num = np.asarray(gt_num)
gt_ana = np.asarray(gt_ana)
plt.plot(t, np.imag( gt_num ) ,'-o')
plt.plot(t, np.imag( gt_ana ),'-<')
plt.show()

plt.plot(t, np.real( gt_num ) ,'-o')
plt.plot(t, np.real( gt_ana ) ,'-<')
plt.show()

# Test interaction w/o femion bath

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import linalg 

# plot setting 
import matplotlib as mpl
from matplotlib import rcParams
from matplotlib import rc

plt.style.use('default')

rcParams[ 'axes.titlesize'] = 25
rcParams[ 'axes.labelsize'] = 25
rcParams[ 'lines.markersize'] = 5
rcParams[ 'xtick.labelsize'] = 22
rcParams[ 'ytick.labelsize'] = 22
rcParams[ 'legend.fontsize'] = 25
rcParams[ 'legend.frameon'] = True

# mpl.rc('font', family='sans-serif')
mpl.rc('text', usetex=True)

mpl.rcParams['text.latex.preamble'] = [
       r'\usepackage{amsmath}',
       r'\usepackage{helvet}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
    
]  

from imp_chain_ham import *
from mps import *
from TEBD import TEBD_class # main solver
from parameter import parameter # parameter

from bath_generator import spectral


In [ ]:

# Set interaction four orbital 
Dmax = 128 # max bound dimension 

num_of_imp = int(4) # four imp
U_hubbard = 2.0 # density interaction
JH         = 1.0 # Hund's


U ={ '12':U_hubbard, '13':U_hubbard - 0.5 * JH, '14':U_hubbard, '23':U_hubbard, '24':U_hubbard - 0.5 * JH, '34':U_hubbard }
J = -0.5 * JH 
ed = np.ones( num_of_imp ) * ( -1.5 * U_hubbard ) #fix half filling 
ed[0] = 0.6
E0 = 0
para = parameter(Dmax, ed, U, J,E0) # import para 

In [ ]:
vl = [0.0]
el = [0.0]
# zero bath 
para.L_total= len(vl) + 1

# --------------------------------------------------------
#                    initialization
# --------------------------------------------------------


# definte Ham
chain_ham = []
for i in range(0,4):
    ham = single_chain_ham(el, vl, 0.0)
    chain_ham.append(ham)

# chain mps
L_bath = len(el)
chain_mpo = []
for i in range(0,4):
    if( i% 2 == 0):
        if_bath_sign = True 
    else:
        if_bath_sign = False
    mps_ = mps(para, L_bath+1, if_bath_sign)
    chain_mpo.append(mps_)
# TEBD solver
tebd_ = TEBD_class(para)
int_gate = int_ham(para,para.tau/2.0)
tebd_.init(para)
tebd_.import_chain_ham(chain_ham)
tebd_.import_int_gate(int_gate, para.tau/2.0)


In [ ]:

N_tau = 500
para.tau = 2.0
En = 9999
int_gate.update_dt(para.tau/2.0)
for i in range(0, N_tau):
    if( i%10 == 0 and i!=0 and para.tau > 0.02):
        para.tau = para.tau/2.0
    
    if( i%2 ==0):
        En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = True)
        print("gnd energy = ",En_new)
        diff = np.abs( En - En_new )
        En = En_new
        if( diff < 1e-8 and i > N_tau/2):
            if( para.tau < 0.02 ):
                print("Solved")
                break 
            else:
                para.tau = para.tau/2.0       
    else:
         En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = False)

In [ ]:
tebd_.save_gnd( )

In [ ]:
tebd_.load_gnd()
imp_ind = 0

Nt = 50
t = []
gt_num = []
para.t = 0.02



tebd_.act_d_dag(imp_ind)

for i in range(0,Nt):
    t.append(i * para.t)
    gt = tebd_.trace_with_d(imp_ind)
    print("------------")
#     gt = chain_mpo[0].trace_()
    print("i, gt",i, gt)
    gt_num.append(gt)
    tebd_.time_evolution(1j * para.t, if_calculate_gnd_en = False)
#     tebd_.one_step_chain_projection(tebd_.chain_mpo[0], tebd_.chain_ham[0], 1j*para.t )
    


In [ ]:
gt_ana = int_gate.d_dag_t_d(t)

In [ ]:
plt.plot(t, gt_num,'-o')
plt.plot(t, gt_ana, '-<')
plt.show()

# Test total

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import linalg 

# plot setting 
import matplotlib as mpl
from matplotlib import rcParams
from matplotlib import rc

plt.style.use('default')

rcParams[ 'axes.titlesize'] = 25
rcParams[ 'axes.labelsize'] = 25
rcParams[ 'lines.markersize'] = 5
rcParams[ 'xtick.labelsize'] = 22
rcParams[ 'ytick.labelsize'] = 22
rcParams[ 'legend.fontsize'] = 25
rcParams[ 'legend.frameon'] = True

# mpl.rc('font', family='sans-serif')
mpl.rc('text', usetex=True)

mpl.rcParams['text.latex.preamble'] = [
       r'\usepackage{amsmath}',
       r'\usepackage{helvet}',    # set the normal font here
       r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
       r'\sansmath'               # <- tricky! -- gotta actually tell tex to use!
    
]  

from imp_chain_ham import *
from mps import *
from TEBD import TEBD_class # main solver
from parameter import parameter # parameter

from bath_generator import spectral


In [ ]:

# Set interaction four orbital 
Dmax = 256 # max bound dimension 

num_of_imp = int(4) # four imp
U_hubbard = 1.0# density interaction
JH         = 0.3 # Hund's


U ={ '12':U_hubbard, '13':U_hubbard - 0.5 * JH, '14':U_hubbard, '23':U_hubbard, '24':U_hubbard - 0.5 * JH, '34':U_hubbard }
J = -0.5 * JH 
ed = np.ones( num_of_imp ) * ( -1.5 * U_hubbard ) #fix half filling 
E0 = 0
para = parameter(Dmax, ed, U, J,E0) # import para



# get bath 
const_spec = spectral()
const_spec.constant_dos(2)
[vl, el, de] = const_spec.get_bath_coeff(2)
[w, rc] = const_spec.get_dos_from_para(vl, el, de, 1.0)

para.L_total= len(vl) + 1

# --------------------------------------------------------
#                    initialization
# --------------------------------------------------------


# definte Ham
chain_ham = []
for i in range(0,4):
    ham = single_chain_ham(el, vl, 0.0)
    chain_ham.append(ham)

# chain mps
L_bath = len(el)
chain_mpo = []
for i in range(0,4):
    if( i% 2 == 0):
        if_bath_sign = True 
    else:
        if_bath_sign = False
    mps_ = mps(para, L_bath+1, if_bath_sign)
    chain_mpo.append(mps_)
# TEBD solver
tebd_ = TEBD_class(para)
int_gate = int_ham(para,para.tau/2.0)
tebd_.init(para)
tebd_.import_chain_ham(chain_ham)
tebd_.import_int_gate(int_gate, para.tau/2.0)


In [ ]:
from ed import ed_solver
import time 
time_st = time.time()
ed_ = ed_solver(para,el,vl) # initalize ed 
ed_.construct_ham()
time_1 = time.time()
print("time = ", (time_1-time_st)/60.0)
ed_.get_eigen_system()
print("time = ", (time.time() -time_1)/60.0)

In [ ]:
N_tau = 500
para.tau = 2.0
En = 9999
int_gate.update_dt(para.tau/2.0)
for i in range(0, N_tau):
    if( i%10 == 0 and i!=0 and para.tau > 0.02):
        para.tau = para.tau/2.0
    
    if( i%2 ==0):
        En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = True)
        print("gnd energy = ",En_new)
        diff = np.abs( En - En_new )
        En = En_new
        if( diff < 1e-8 and i > N_tau/2):
            if( para.tau < 0.02 ):
                print("Solved")
                break 
            else:
                para.tau = para.tau/2.0       
    else:
         En_new = tebd_.time_evolution(para.tau, if_calculate_gnd_en = False)
            
tebd_.save_gnd( )

In [ ]:
print("Gnd energy = ", ed_.get_gnd_energy())
print(ed_.eg[0:3])

In [ ]:
tebd_.load_gnd()
imp_ind = 3

Nt = 10
t = []
gt_num = []
para.t = 0.02



tebd_.act_d_dag(imp_ind)

for i in range(0,Nt):
    t.append(i * para.t)
    gt = tebd_.trace_with_d(imp_ind)
    print("------------")
#     gt = chain_mpo[0].trace_()
    print("i, gt",i, gt)
    gt_num.append(gt)
    tebd_.time_evolution(1j * para.t, if_calculate_gnd_en = False)
#     tebd_.one_step_chain_projection(tebd_.chain_mpo[0], tebd_.chain_ham[0], 1j*para.t )
    

In [ ]:
gt_ana = ed_.get_c_t_cdag_gnd(t,0)

In [ ]:
plt.plot(t, np.real(gt_num), '-o')
plt.plot(t, np.real(gt_ana),'-<')
plt.show()
plt.plot(t, np.imag(gt_num), '-o')
plt.plot(t, np.imag(gt_ana),'-<')
plt.show()

In [ ]:
tebd_.load_gnd()
imp_ind = 3

Nt = 10
t = []
gt_num = []
para.t = 0.02



tebd_.act_d(imp_ind)

for i in range(0,Nt):
    t.append(i * para.t)
    gt = tebd_.trace_with_d_dag(imp_ind)
    print("------------")
#     gt = chain_mpo[0].trace_()
    print("i, gt",i, gt)
    gt_num.append(gt)
    tebd_.time_evolution(1j * para.t, if_calculate_gnd_en = False)
#     tebd_.one_step_chain_projection(tebd_.chain_mpo[0], tebd_.chain_ham[0], 1j*para.t )
gt_ana = ed_.get_cdag_t_c_gnd(t,0)  

In [ ]:
plt.plot(t, np.real(gt_num), '-o')
plt.plot(t, np.real(gt_ana),'-<')
plt.show()
plt.plot(t, np.imag(gt_num), '-o')
plt.plot(t, np.imag(gt_ana),'-<')
plt.show()